In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime as dt
%matplotlib inline

import requests
import json
import calendar
import time

In [51]:
tweets_df = pd.read_pickle('./Data/Cleaned/processed_data.pkl')
xrp = pd.read_pickle('./Data/Cleaned/xrp_price_data_hourly.pkl')

In [52]:
tweets_df['datetime'] = tweets_df['datetime'].apply(lambda x : x.to_pydatetime())

In [53]:
def cielDatetime(d):
    delta = 3600-((d.minute*60+d.second)%3600)
    
    td = dt.timedelta(seconds=delta)
    return d.to_pydatetime()+td

In [54]:
tweets_df['rounded_dateTime'] = tweets_df['datetime'].apply(lambda t : cielDatetime(t))

In [55]:
tweets_df['timeId'] = tweets_df['rounded_dateTime'].apply(lambda t : time.mktime(t.timetuple()))

In [56]:
xrpDelta = xrp.set_index('time')
xrpDelta.sort_index(inplace=True)
xrpDelta = xrpDelta.diff()

In [57]:
prev = xrp.set_index('time')
temp = pd.concat([pd.Series([np.nan]), xrp['close']]).reset_index(drop=True)
prev['close'] = temp[0:298].values

In [58]:
xrpDelta['pDelta'] = xrpDelta['close']/prev['close']
xrpDelta['pDelta'][0] = 0.027

In [59]:
xrpDelta = xrpDelta.reset_index('time')
xrpDelta = xrpDelta[['time', 'pDelta']]
xrpDelta['timeId'] = xrpDelta['time'].apply(lambda t : time.mktime(t.timetuple()))

def deltaConvert(d):
    if d > 0.005:
        return 'positive'
    elif d < -0.005:
        return 'negative'
    else:
        return 'neutral'

xrpDelta['change'] = xrpDelta['pDelta'].apply(lambda price : deltaConvert(price))


In [60]:
xrpDelta.head()

,time,pDelta,timeId,change
0,2018-04-13 04:00:00,0.027000,1.523606e+09,positive
1,2018-04-13 05:00:00,-0.014048,1.523610e+09,negative
2,2018-04-13 06:00:00,0.014717,1.523614e+09,positive
3,2018-04-13 07:00:00,0.000926,1.523617e+09,neutral
4,2018-04-13 08:00:00,0.059349,1.523621e+09,positive


In [61]:
fullData = pd.merge(tweets_df, xrpDelta, how='left', on='timeId')
fullData = fullData[np.isfinite(fullData['pDelta'])]

In [62]:
fullData.head()

,ID,datetime,is_reply,is_retweet,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,has_media,medias,rounded_dateTime,timeId,time,pDelta,change
0,9.885302e+17,2018-04-23 17:29:08,0.0,0.0,0.0,0.0,0.0,New post (Could Ripple XRP Really Reach $1...,/ExcelVisibility/status/988530235493748736,815322320,ExcelVisibility,NaN,NaN,2018-04-23 18:00:00,1.524521e+09,2018-04-23 18:00:00,0.002178,neutral
1,9.866292e+17,2018-04-18 11:35:08,0.0,0.0,0.0,0.0,0.0,Long/Short Bitcoin swings with up to 100x Leve...,/KerriHermanx/status/986629205856223233,956303409065414656,KerriHermanx,NaN,NaN,2018-04-18 12:00:00,1.524067e+09,2018-04-18 12:00:00,0.011943,positive
2,9.847927e+17,2018-04-13 09:57:41,0.0,0.0,0.0,0.0,0.0,"Did you know that you can buy, sell, store, co...",/coindirectcom/status/984792745666580481,915453520924545025,coindirectcom,1.0,[https://t.co/h1CTJHGwPs],2018-04-13 10:00:00,1.523628e+09,2018-04-13 10:00:00,-0.004874,neutral
3,9.863800e+17,2018-04-17 19:05:02,0.0,0.0,0.0,0.0,0.0,1 Ripple = 0.6589 USD. Ripple has changed ...,/RippleMarket/status/986380040517701633,848989770923601922,RippleMarket,NaN,NaN,2018-04-17 20:00:00,1.524010e+09,2018-04-17 20:00:00,0.007330,positive
4,9.866047e+17,2018-04-18 09:57:42,0.0,0.0,0.0,0.0,0.0,How Is #Ripple Different From All Other #Cryp...,/Matthix191/status/986604686856933376,916641996823031808,Matthix191,1.0,[https://t.co/xF5l49lz8P],2018-04-18 10:00:00,1.524060e+09,2018-04-18 10:00:00,-0.014478,negative


In [63]:
fullData.to_pickle('./Data/Cleaned/temporary_processed_data.pkl')